# Extended MNIST

L'obiettivo del progetto è il riconoscimento e la conseguente classificazione di immagini rappresentati lettere scritte a mano. Il dataset fornito è composto da 80000 immagini di stessa dimensione (28x28), rappresentate in scala di grigi. Non viene fatta distizione tra lettere maiuscole e minuscole, quindi le classi da individuare sono 26.

# Approccio alla soluzione

Verranno utilizzati più modelli per la soluzione al problema, partendo da un modello di base (Logistic Regression) fino ad arrivare a modelli più complessi (Neural Network). Le prestazioni di ogni modello verrano giudicate in base all'accuratezza ed al tempo di esecuzione.  

In [5]:
#import gc

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras_tuner as kt

from time import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#utility functions

#time monitor
def elapsed_time(start, end):
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Time spent training: {:0>2}:{:0>2}:{:0>2}".format(int(hours),int(minutes),int(seconds)))

#accuracy printer
def print_accuracy_scores(train_labels, test_labels, predictions_train_model, predictions_test_model):
    acc_train = accuracy_score(train_labels, predictions_train_model)
    acc_test = accuracy_score(test_labels, predictions_test_model)
    print('Training set accuracy:   {:.3f}'.format(acc_train))
    print('Test set accuracy:       {:.3f}'.format(acc_test))

#confusion matrix printer
def plot_confusion_matrix(train_labels, predictions_train):
    plt.figure(figsize=(12,6))
    disp = ConfusionMatrixDisplay.from_predictions(train_labels, predictions_train, normalize='true', cmap='Blues')
    disp.figure_.suptitle("Confusion Matrix")
    plt.grid(None)
    plt.show()

I dati sono stati divisi in 3 set: train (70%), validation (15%) e test set(15%).

In [6]:
#reading data from csv and split the data in train, validation and test sets

df = pd.read_csv("emnist-letters.csv")
X = df.iloc[:,1:].to_numpy().reshape(-1, 28, 28, order="F")
y = df.iloc[:,0].to_numpy()-1
print(y)

train_images, test_val_images, train_labels, test_val_labels = train_test_split(X,y, test_size=0.3, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(test_val_images, test_val_labels, test_size=0.5, random_state=42)

X = np.concatenate((train_images, val_images), axis=0)
y = np.concatenate((train_labels, val_labels), axis=0)
split_index = [-1]*len(train_images) + [0]*len(val_images)
pds = PredefinedSplit(test_fold = split_index)


[ 6 15 14 ...  0 22 11]


# Softmax Regression

Prima di applicare ogni modello, attraverso una GridSearch si individua la migliore combinazione dei parametri del modello scelto. A causa dei tempi di esecuzioni troppo elevati, non è stato possibile usare una cross-validation, ma una semplice "1fold".

Nel caso della Softmax Regression, i paramteri soggetti a tuning sono:
 - C (il fattore di regolarizzazione)
  - max_iter (il numero massimo di iterazioni del solver).

In [ ]:
params = {"C":[0.1, 0.01, 0.001],
         "max_iter":[100, 200, 500, 1000]}
clf = GridSearchCV(LogisticRegression(multi_class='multinomial', n_jobs=-1), params, scoring='accuracy', return_train_score=True, cv=pds)

time_start = time()

clf.fit(X, y)

time_end = time()
elapsed_time(time_start, time_end)

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_score_

In [ ]:
log_reg = LogisticRegression(multi_class='multinomial', n_jobs=-1, C=0.01, max_iter=1000)
time_start = time()
log_reg.fit(train_images, train_labels)
time_end = time()
elapsed_time(time_start, time_end)

predictions_test = log_reg.predict(test_images)
predictions_train = log_reg.predict(train_images)

print_accuracy_scores(train_labels, test_labels, predictions_train, predictions_test)
plot_confusion_matrix(train_labels, predictions_train)

# Random forest

Nella Random Forest, i paramteri soggetti a tuning sono:
- n_estimators (il numero di alberi)
- criterion (funzione di impurità)
- min_samples_split (il numeri minimo di elementi di un nodo affinché possa essere partizionato)
- max_depth (profondità dell'albero).

La suddivisione del dataset in train, validatione test set rimane la stessa: 70% train set, 15% test set e 15% validation set.


In [ ]:
n_estimators =[50, 100, 200, 500, 1000] 
criterion = ["gini", "entropy"]
min_samples_split = [2, 4, 8, 16, 32, 64, 128, 256, 1024, 2048]
max_depth = [1, 5, 10, 20, 45, 50]

def build_random_forest(hp):
    model = RandomForestClassifier(
        n_jobs=-1, 
        random_state=42,
        n_estimators=hp.Choice("n_estimators", n_estimators),
        criterion=hp.Choice("criterion", criterion),
        min_samples_split=hp.Choice("min_samples_split", min_samples_split),
        max_depth=hp.Choice('max_depth', max_depth))
    return model

In [ ]:
rf_trials=80
tuner = kt.tuners.SklearnTuner(
    oracle=kt.oracles.RandomSearchOracle(objective=kt.Objective('score', 'max'),max_trials=rf_trials, seed=42),
    scoring='accuracy',
    hypermodel= build_random_forest,
    cv=pds,
    project_name='tuners/random_forest')

tuner.search(X, y)
#gc.collect()
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\nBest criterion:        ", best_hps.get("criterion"))
print("Best max_depth:          ", best_hps.get("max_depth"))
print("Best n_estimators:       ", best_hps.get("n_estimators"))
print("Best min_samples_split:  ", best_hps.get("min_samples_split"))

In [ ]:
model = RandomForestClassifier(n_jobs=-1, criterion=best_hps.get("criterion"), 
                               max_depth=best_hps.get("max_depth"), n_estimators=best_hps.get("n_estimators"), min_samples_split=best_hps.get("min_samples_split"))
time_start = time()
model.fit(train_images, train_labels)
time_end = time()
elapsed_time(time_start, time_end) 
print("")

predictions_test = model.predict(test_images)
predictions_train = model.predict(train_images)

print_accuracy_scores(train_labels, test_labels, predictions_train, predictions_test)
plot_confusion_matrix(train_labels, predictions_train)

# Reti neurali convoluzionali (CNN)

Come ultimo modello si è scelto di utilizzare una CNN. Partendo da una rete semplice, questa verrà resa più complessa con l'aggiunta di nuovi layer ed ad ogni modello verrà applicato il tuning degli iperparametri.

# Modello 0
